# Numerical Benchmarks

In [1]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from lindblad_channels import *
from quantum_tools import *
from experimental import *
from spam import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Recover SPAM and Map

### Generate True Model, Full POVM and Inital

In [2]:
def generate_spam_benchmark(n=3, c1=1, c2=1):
    d = 2**n

    init_target = InitialState(d, c=c1)
    povm_target = POVM(d, c=c2)

    spam_target = SPAM(init = init_target,
                       povm = povm_target)
    
    return spam_target


def generate_spam_data(spam_target, N_spam=None, shots=1024):
    n = int(np.log2(spam_target.d))
    inputs_spam, _ = generate_pauliInput_circuits(n)
    N_spam = inputs_spam.shape[0]

    state = tf.repeat(spam_target.init.init[None,:,:], N_spam, axis=0)
    targets_spam = measurement(state, U_basis = inputs_spam, povm = spam_target.povm.povm)

    #add noise
    targets_spam = add_shot_noise(targets_spam, shots = shots)
    return inputs_spam, targets_spam


def generate_map_data(channel_target, spam_target, N_map=None, shots=1024, grid=True, batch_size=1000):
    n = int(np.log2(channel_target.d))
    inputs_map, _ = generate_pauli_circuits(n = n, 
                                            circuit_target=None,  
                                            N=N_map,
                                            grid=grid)
    U_prep, U_basis = inputs_map

    N_map = U_prep.shape[0]
    
    targets_map = []
    for batch_num in range(N_map//batch_size + 1):

        U_prep_batch = U_prep[batch_num*batch_size:(batch_num+1)*batch_size]
        U_basis_batch = U_basis[batch_num*batch_size:(batch_num+1)*batch_size]
        batch_size_current = U_prep_batch.shape[0]

        state = tf.repeat(tf.expand_dims(spam_target.init.init, axis=0), batch_size_current, axis=0)
        state = apply_unitary(state, U_prep_batch)
        state = channel_target.apply_channel(state)
        probs = measurement(state, U_basis_batch, spam_target.povm.povm)
        probs = add_shot_noise(probs, shots = shots)
        targets_map.append(probs)
    
    targets_map = tf.concat(targets_map, axis=0)

    return inputs_map, targets_map


def model_pipeline(channel_target = None, shots=1024, N_map = None, num_iter_spam=None, num_iter_map=None, verbose=False):
    # Make Benchmark
    #################################################################################
    n = 4
    d = 2**n
    
    spam_target = generate_spam_benchmark(n=n, c1=0.9, c2=0.8)
    inputs_spam, targets_spam = generate_spam_data(spam_target, N_spam=None, shots=shots)

    inputs_map, targets_map = generate_map_data(channel_target, spam_target, N_map=N_map, shots=shots)
    #################################################################################

    # Fit Models
    #################################################################################
    spam_model = SPAM(init = InitialState(d),
                      povm = CorruptionMatrix(d),
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    spam_model.pretrain(500, verbose=False)

    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = num_iter_map,
                     verbose = False,
                    )

    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                               rank = d**2,
                                               spam = spam_model,
                                              ),
                            loss_function = ProbabilityMSE(),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            logger = Logger(loss_function = ProbabilityMSE(),
                                            loss_function_val = channel_fidelity_loss,
                                            sample_freq = 100),
                           )

    model.train(inputs = inputs_map,
                targets = targets_map,
                inputs_val = None,
                targets_val = [channel_target],
                num_iter = num_iter_map,
                N = 500,
                verbose = verbose,
                )
    #################################################################################
    model.optimizer = None
    spam_model.optimizer = None
    model.inputs = None
    model.targets = None
    model.inputs_val = None
    model.targets_val = None

    return model

## Fit Models

In [3]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 4
d = 2**n

channel_target1 = CompactLindbladMap(d, 1, 1, 0.1)
model1 = model_pipeline(channel_target1, 
                        shots=1024, 
                        N_map=10000-6**n, 
                        num_iter_spam = 1000,
                        num_iter_map = 2000,
                        verbose=True)

channel_target2 = CompactLindbladMap(d, 16, 10000, 0.001)
model2 = model_pipeline(channel_target2, 
                        shots=1024, 
                        N_map=10000-6**n, 
                        num_iter_spam = 1000,
                        num_iter_map = 2000,
                        verbose=True)

channel_target3 = CompactLindbladMap(d, 16, 100, 0.001)
model3 = model_pipeline(channel_target3, 
                        shots=1024, 
                        N_map=10000-6**n, 
                        num_iter_spam = 1000,
                        num_iter_map = 2000,
                        verbose=True)

channel_target4 = CompactLindbladMap(d, 8, 1, 0.01)
model4 = model_pipeline(channel_target4, 
                        shots=1024, 
                        N_map=10000-6**n, 
                        num_iter_spam = 1000,
                        num_iter_map = 2000,
                        verbose=True)


saver([[channel_target1, channel_target2, channel_target3, channel_target4],
       [model1, model2, model3, model4]], "data/"+"atypical_maps_benchmark.model")

  0%|          | 0/2000 [00:00<?, ?it/s]

0.15239112138286287 -0.062276727520169245
0.06894455839352565 -0.18953657421532247
0.032219157084771764 -0.4239175231881091
0.019700362457262685 -0.6033991722497707
0.016792361320542248 -0.700728961734525
0.01567176367624108 -0.7561999113461048
0.015118092727943037 -0.790335726176199
0.014227069967910061 -0.8122485477449919
0.014315818368274976 -0.8284813521537212
0.014296914440136789 -0.8399177009461666
0.014164691900754573 -0.8485673976377013
0.013811531860313156 -0.854306848077786
0.01420502632927155 -0.8596635627204989
0.014349934438074747 -0.8632305796047993
0.014061616387647843 -0.8662619728851423
0.013883498003471306 -0.8685974253183747
0.013572256629052305 -0.8706930705514816
0.014707384579405442 -0.8720480169231518
0.013816358981590572 -0.8729951271187456
0.013893581295231298 -0.87406322773848
0.014100590194416356 -0.8745911477409736


  0%|          | 0/2000 [00:00<?, ?it/s]

0.18826639066234252 -0.3160505878501399
0.08944023300559473 -0.5633122033622049
0.017129100625223232 -0.7962626706694079
0.015339934508668246 -0.8232376760553718
0.014503945879748285 -0.8301067372521782
0.014108383801399511 -0.8323599163884814
0.01398377126871008 -0.8313309407804389
0.013616883601694628 -0.828141198448411
0.01349597089729577 -0.8232808686381049
0.013069682228729545 -0.8178119903837974
0.013384512267683717 -0.8110699713047762
0.013332579353756077 -0.8044599212535064
0.012898703203501604 -0.7975192831524808
0.01246227424086931 -0.79079690905647
0.013145468843978273 -0.7840492824327382
0.012860060605380341 -0.7772397163827139
0.012786821790993297 -0.7711205608402094
0.012876360503217658 -0.7651561429913264
0.01288731882846652 -0.7594276609015009
0.012543074536047092 -0.7540229253324844
0.012854240756301746 -0.7488424114715139


  0%|          | 0/2000 [00:00<?, ?it/s]

0.34450560658895224 -0.16695544932299508
0.14319971981874766 -0.280561091142401
0.03828977565737105 -0.4686884577394987
0.021331683174167664 -0.5881236012645238
0.017701032056721536 -0.6466113556219107
0.016052749692052247 -0.6850940402865374
0.01537127996207539 -0.7115992181833835
0.014633319610789662 -0.7318410551956246
0.014320175694487815 -0.7472046217006326
0.013814963496508608 -0.7594327510303338
0.014050779496808755 -0.7693221753026013
0.013714474233844132 -0.7771366809550178
0.013782634139632915 -0.7835344119680605
0.013555809386837354 -0.7882816766722356
0.013602805763511725 -0.7922326809686745
0.01323335140398813 -0.7950243324670963
0.013490443283401916 -0.7970411273819937
0.013020649555353099 -0.7987037603646846
0.01346622711445595 -0.7996224186960259
0.013281867864613122 -0.800176394809549
0.01349366845259583 -0.8005948517128905


  0%|          | 0/2000 [00:00<?, ?it/s]

0.035590623726822826 -0.5460646828358755
0.018588781221051606 -0.6278142235156671
0.01569289805890618 -0.6817881165079711
0.013673531867204491 -0.7107791940490888
0.013253212756093594 -0.7245607128166283
0.012954399688325378 -0.7298752714786757
0.012658405804052728 -0.7298698966169042
0.01212164810414355 -0.7264598165213708
0.012199366436246304 -0.720928679038671
0.012329267006118978 -0.7141688969973246
0.011654758826173502 -0.7074349703810816
0.011795617655301545 -0.7000852815469395
0.011646173466857219 -0.692979248435242
0.011714413928766257 -0.6861392657671489
0.012084428942067241 -0.6791142057816756
0.011933014010753189 -0.6726807021938503
0.011669481242819312 -0.6661559053431809
0.012074187360881089 -0.660217877773428
0.011642499764670597 -0.6548207658471382
0.011755969784327835 -0.6491557539957319
0.011850366706920525 -0.6444123437621659


## Sample and Noise Sensitivity

In [ ]:
n = 3
d = 2**n
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

channel_target = CompactLindbladMap(d, 1, 1, 0.1)
model_ns1 = model_pipeline(channel_target, 
                           shots=256, 
                           N_map=256, 
                           num_iter_spam = 4000,
                           num_iter_map = 4000,
                           verbose=True)


model_ns2 = model_pipeline(channel_target, 
                           shots=1024, 
                           N_map=1024, 
                           num_iter_spam = 4000,
                           num_iter_map = 4000,
                           verbose=True)

model_ns3 = model_pipeline(channel_target, 
                           shots=4096, 
                           N_map=4096, 
                           num_iter_spam = 4000,
                           num_iter_map = 4000,
                           verbose=True)

saver([[channel_target],
       [model_ns1, model_ns2, model_ns3]], "data/"+"atypical_maps_robustness.model")

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

0.25255740329077125 -0.09790789532063889
0.016885521111175632 -0.7222070780249391
0.015452065602110869 -0.760714612585643
0.01531860940478661 -0.7641966034332381
0.015297122467690733 -0.7642737530500818
0.015292475955475414 -0.7639739577406761
0.015291226336929678 -0.7637341865100247
0.015290815872548092 -0.7635822063635365
0.015290654856823335 -0.7634848393048708
0.01529058398287721 -0.7634169420276298
0.015290549716756884 -0.7633666378726003


  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

0.27708061330793593 -0.09534488348365604
0.007335178428198955 -0.848321899569145
0.006000436057100031 -0.9126782156148544
0.006099827470396115 -0.9254013088917373
0.005813236066433264 -0.9287196248575446
0.005922847688239885 -0.9301088602481353
0.005848523504409336 -0.9302654758922971
0.005760523656058943 -0.9307514814209986
0.005870826412870416 -0.9307074604315804
0.005738973434337544 -0.9305317511651956
0.005925678601175935 -0.9304364126511478


  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

0.23703334880839866 -0.09578037331174998
0.0029864486414783366 -0.8720496424790909
0.0019357776372289142 -0.9424086891684207
0.0018231443703856038 -0.9612835439292674
0.0017617372507687896 -0.9683585702097184
0.0017684640141524685 -0.9713771371291521
0.0017746410494197695 -0.972789048951825
0.001789841067956938 -0.9735358872083617
0.0017617855700840378 -0.9736753982369741
0.0016946056043024946 -0.973672610133213
0.0017370244957168452 -0.9738120782717761
